In [1]:
# Import dependencies
import pandas as pd
from pyxlsb import open_workbook

# Extract Data Into DataFrames

### Data 1: Irena.org Dataset on Electricity Installed Capacity (MW)

In [2]:
# Import data file
irena_eic_file = "data/IRENA_Stats_Tool.xlsb"

irena_eic_df = []
with open_workbook (irena_eic_file) as wb:
     with wb.get_sheet("Data") as sheet:
        for row in sheet.rows():
            irena_eic_df.append([item.v for item in row])

irena_eic_df = pd.DataFrame(irena_eic_df[8:], columns=irena_eic_df[5])
irena_eic_df.head()

,Region,Country,ISO Code,RE or Non-RE,Group Technology,Technology,Poducer Type,Year,Electricity Installed Capacity (MW),,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Africa,Algeria,DZA,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,2018.0,21308.0,None,...,None,None,None,None,None,None,None,None,None,None
1,Africa,Algeria,DZA,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,2019.0,23508.0,None,...,None,None,None,None,None,None,None,None,None,None
2,Africa,Algeria,DZA,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,2018.0,228.0,None,...,None,None,None,None,None,None,None,None,None,None
3,Africa,Algeria,DZA,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,2019.0,228.0,None,...,None,None,None,None,None,None,None,None,None,None
4,Africa,Algeria,DZA,Total Renewable,Solar energy,Concentrated solar power,On-grid,2018.0,25.0,None,...,None,None,None,None,None,None,None,None,None,None


### Data 2: bp.com Dataset on Energy Consumption (Exajoules)

In [3]:
# Import data file
bp_pe_file = "data/bp-stats-review-2020-all-data.xlsx"

bp_pe = pd.read_excel(bp_pe_file, "Primary Energy Consumption", engine="openpyxl")

# Rename the columns with value from the second row
bp_pe_rename = bp_pe.rename(columns=bp_pe.iloc[1])
bp_pe_rename.head()

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN


# Transform Data

### Data 1: Irena.org Dataset on Electricity Installed Capacity (MW)

In [4]:
# Get coloumns 
irena_eic_df.columns

Index([                             'Region',
                                   'Country',
                                  'ISO Code',
                              'RE or Non-RE',
                          'Group Technology',
                                'Technology',
                              'Poducer Type',
                                      'Year',
       'Electricity Installed Capacity (MW)',
                                          '',
                                          '',
                                          '',
                                          '',
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None],
      dtype='object')

In [5]:
# Extract wanted columns
eic_cols = ['Region','Country','Year','RE or Non-RE','Group Technology','Technology','Poducer Type','Electricity Installed Capacity (MW)']
eic_countries = irena_eic_df[eic_cols].copy()
eic_countries.head()

,Region,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
0,Africa,Algeria,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,21308.0
1,Africa,Algeria,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,23508.0
2,Africa,Algeria,2018.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.0
3,Africa,Algeria,2019.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.0
4,Africa,Algeria,2018.0,Total Renewable,Solar energy,Concentrated solar power,On-grid,25.0


In [6]:
# Drop rows with empty values
eic_countries.dropna()

,Region,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
0,Africa,Algeria,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,21308.00
1,Africa,Algeria,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,23508.00
2,Africa,Algeria,2018.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.00
3,Africa,Algeria,2019.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.00
4,Africa,Algeria,2018.0,Total Renewable,Solar energy,Concentrated solar power,On-grid,25.00
...,...,...,...,...,...,...,...,...
2968,South America,Venezuela,2019.0,Total Renewable,Solar energy,Off-grid Solar photovoltaic,Off-grid,3.00
2969,South America,Venezuela,2018.0,Total Renewable,Solar energy,On-grid Solar photovoltaic,On-grid,1.00
2970,South America,Venezuela,2019.0,Total Renewable,Solar energy,On-grid Solar photovoltaic,On-grid,2.32
2971,South America,Venezuela,2018.0,Total Renewable,Wind energy,Onshore wind energy,Off-grid,71.28


#### Extracting data from my selected three countries: China, Germany, and United States

In [7]:
# Extracting China's data
eic_china = eic_countries[eic_countries["Country"]=="China"]
eic_china.set_index("Region", inplace=True)
eic_china.head()

,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Region,,,,,,,
Asia,China,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,1126270.000
Asia,China,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,1168004.147
Asia,China,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,44660.000
Asia,China,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,48740.000
Asia,China,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,4575.000


In [8]:
# Extracting Germany's data
eic_germany = eic_countries[eic_countries["Country"]=="Germany"]
eic_germany.set_index("Region", inplace=True)
eic_germany.head()

,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Region,,,,,,,
Europe,Germany,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,91415.0
Europe,Germany,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,91991.0
Europe,Germany,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,10799.0
Europe,Germany,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,10799.0
Europe,Germany,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,2332.0


In [9]:
# Extracting USA's data
eic_usa = eic_countries[eic_countries["Country"]=="United States"]
eic_usa.set_index("Region", inplace=True)
eic_usa.head()

,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Region,,,,,,,
North America,United States,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,750619.0
North America,United States,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,743307.0
North America,United States,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,99629.0
North America,United States,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,98266.3
North America,United States,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,3292.5


### Data 2: bp.com Dataset on Energy Consumption (Exajoules)

In [31]:
# Extract wanted columns
bp_ec_countries = bp_pe_rename.copy()
bp_ec_countries.head()

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN


In [34]:
bp_ec_countries ["2019_share"] = bp_ec_countries.iloc [:,58]
bp_ec_countries ["2019_growth_rate_per_annum"] = bp_ec_countries.iloc [:,56]
bp_ec_countries ["2019_year"] = bp_ec_countries.iloc[:,55]
bp_ec_countries

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2018.0,2019.0,2019,2008-18,2019,NaN,NaN,2019_share,2019_growth_rate_per_annum,2019_year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN,Share,Growth rate per annum,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN,2019,2019,2019.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN,0.0243434,-0.00944118,14.214058
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN,0.0132227,-0.0143718,7.720742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
ls = list(bp_ec_countries.columns)
ls 

['Exajoules',
 1965.0,
 1966.0,
 1967.0,
 1968.0,
 1969.0,
 1970.0,
 1971.0,
 1972.0,
 1973.0,
 1974.0,
 1975.0,
 1976.0,
 1977.0,
 1978.0,
 1979.0,
 1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 2019.0,
 2019,
 '2008-18',
 2019,
 nan,
 nan,
 '2019_share',
 '2019_growth_rate_per_annum',
 '2019_year']

In [39]:
# Get coloumns 
ls = list(bp_ec_countries.columns)
columns=['Exajoules',
 1965.0,
 1966.0,
 1967.0,
 1968.0,
 1969.0,
 1970.0,
 1971.0,
 1972.0,
 1973.0,
 1974.0,
 1975.0,
 1976.0,
 1977.0,
 1978.0,
 1979.0,
 1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 '2019_year', 
 '2019_growth_rate_per_annum',
 '2008-18',
 '2019_share']

In [40]:
bp_ec_countries = bp_ec_countries[columns]
bp_ec_countries

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-18,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2013.000000,2014.000000,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.881791,14.031964,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.738309,7.697182,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
bp_ec_countries_rename = bp_ec_countries.rename(columns={bp_ec_countries.columns[57]: "2008-2018_growth_rate_per_annum"})
bp_ec_countries_rename

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-2018_growth_rate_per_annum,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2013.000000,2014.000000,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.881791,14.031964,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.738309,7.697182,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
